# Function Calling

In [12]:
import os
from openai import OpenAI
import requests
from datetime import datetime
import json


token = os.environ["OPENAI_API_KEY"]
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4o"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)
# Get the current date and time
current_date = datetime.now().strftime("%Y-%m-%d")
current_time = datetime.now().strftime("%H:%M:%S")

def get_weather(latitude, longitude):
    url = "https://ai-weather-by-meteosource.p.rapidapi.com/time_machine"
    querystring = {"lat":latitude,"lon":longitude,"date":current_date, "units":"auto"}
    headers = {
        "x-rapidapi-key": os.environ["RAPID_API_KEY"],
        "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    return response.json()


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the historical weather in a given location by latitude and longitude",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "Latitude of the location"
                    },
                    "longitude": {
                        "type": "number",
                        "description": "Longitude of the location"
                    }
                },
                "required": ["latitude", "longitude"]
            }
        }
    }
]

prompt = "What's the weather like in Colombo today?"
input_messages = [{"role": "user", "content": prompt}]


response = client.chat.completions.create(
    model="openai/gpt-4o",
    messages=input_messages,
    tools=tools,
    tool_choice="auto",
    temperature=0.7,
    max_tokens=4096,
    top_p=1
)

tool_call = response.choices[0].message.tool_calls[0]
tool_args = tool_call.function.arguments
args = json.loads(tool_args)

results = get_weather(args["latitude"], args["longitude"])


input_messages_short = [
    input_messages[0],  # original user message
    {
        "role": "assistant",
        "tool_calls": [tool_call],  
    },
    {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": str({
            "lat": results.get("lat"),
            "lon": results.get("lon"),
            "timezone": results.get("timezone"),
            "units": results.get("units"),
            "current": results.get("data", [{}])[0]  # just the first data point
        })
    }
]

response = client.chat.completions.create(
    model="openai/gpt-4o",
    messages=input_messages_short,
    tools=tools,
    tool_choice="auto",
    temperature=0.4,
    max_tokens=4096,
    top_p=1
)

print(response.choices[0].message.content)


The weather in Colombo today is rainy with a temperature of 25.7°C, but it feels like 29.5°C. The wind is coming from the south-southwest at a speed of 2.2 km/h, with gusts up to 4.8 km/h. The humidity is quite high at 91%, and the cloud cover is 100%. There has been 0.6 mm of rainfall so far.


### Output

Today's weather in Colombo is rainy with a temperature of 25.7°C, though it feels like 29.5°C. The wind is coming from the south-southwest (SSW) at a speed of 2.2 km/h with gusts up to 4.8 km/h. The humidity is high at 91%, and there's been light rainfall with a total of 0.6 mm. It's fully cloudy with 100% cloud cover.